<a href="https://colab.research.google.com/github/marcusflygar1-hash/project_testing/blob/main/project_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OBS! README

For clarification the GridSearch part of this project was created through ai of AI.

Also be wary of what code you run in which order.
If you want to see  the results while using PCA, DO NOT run the entire notebook at once, you have to pick and chose what parts you run. Same goes for all types of functions and especially if you want to garner the results from the supervised labeling that I have done my self.

#  Setup & Imports

In [ ]:
# importing the neccesary libraries for the project.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import auc,roc_auc_score,roc_curve, accuracy_score, confusion_matrix, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, mean_squared_log_error, r2_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

Nya features att addera:

Veckodagar, månad & ?


Gör modellen mer robust, genom att normalisera portal datan?

GridSearchCV? ..
För rapporten: hade video sensorer varit ännu bättre för att se om det sker "micro-congestions" vid vissa tillfällen, påverkat av förare som tvärbromsar, byter fil utan at blinka etc..


# PCA Function

In [ ]:
from sklearn.decomposition import PCA

def pca_anomaly_labeling(df, n_components=3, contamination=0.1):
    """
    Unsupervised anomaly detection using PCA reconstruction error.
    Marks high reconstruction errors (low speeds, unusual patterns) as congestion = 1.
    """
    df = df.copy().dropna(subset=["SPEED_MS_AVG", "FLOW"])
    features = ["SPEED_MS_AVG", "FLOW"]

    # Normalize before PCA
    X = StandardScaler().fit_transform(df[features])

    pca = PCA(n_components=n_components, random_state=42)
    X_pca = pca.fit_transform(X)
    X_reconstructed = pca.inverse_transform(X_pca)

    # Compute reconstruction error
    errors = np.mean((X - X_reconstructed) ** 2, axis=1)
    df["pca_error"] = errors

    # Determine threshold based on contamination fraction (top X% anomalies)
    threshold = np.percentile(errors, 100 * (1 - contamination))
    df["pca_congestion"] = (df["pca_error"] > threshold).astype(int)

    return df, threshold


# Clustering Function

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

def kmeans_congestion_labeling(df, n_clusters=3):
    """
    Unsupervised labeling using KMeans clustering on SPEED_MS_AVG and FLOW.
    Assigns the cluster with the lowest mean speed as 'congested' (label=1).
    """
    df = df.copy().dropna(subset=["SPEED_MS_AVG", "FLOW"])
    features = ["SPEED_MS_AVG", "FLOW"]

    # Scale the features (important for KMeans)
    X = StandardScaler().fit_transform(df[features])

    # Fit KMeans
    km = KMeans(n_clusters=n_clusters, random_state=42, n_init=20)
    df["cluster"] = km.fit_predict(X)

    # Find which cluster corresponds to congestion
    cluster_speed_means = df.groupby("cluster")["SPEED_MS_AVG"].mean()
    congested_cluster = cluster_speed_means.idxmin()  # lowest mean speed

    # Assign congestion label: 1 = congested, 0 = non-congested
    df["cluster_congestion"] = (df["cluster"] == congested_cluster).astype(int)

    return df, km, congested_cluster


#DBSCAN Method

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

def dbscan_congestion_labeling(df, eps=0.3, min_samples=30):
    """
    Unsupervised congestion labeling using DBSCAN clustering on SPEED_MS_AVG and FLOW.
    Marks sparse/noise points (label = -1) as 'congested' = 1.
    """
    df = df.copy().dropna(subset=["SPEED_MS_AVG", "FLOW"])
    features = ["SPEED_MS_AVG", "FLOW"]

    # Scale features
    X = StandardScaler().fit_transform(df[features])

    # Fit DBSCAN
    db = DBSCAN(eps=eps, min_samples=min_samples)
    df["db_cluster"] = db.fit_predict(X)

    # DBSCAN assigns -1 to points not belonging to any cluster (noise)
    df["db_congestion"] = (df["db_cluster"] == -1).astype(int)

    return df, db


# Data prepping

# Data Loading & Preprocessing

In [ ]:
tds = pd.read_csv(r"C:\Users\playm\OneDrive\Skrivbord\Labbar - Transport\dataset_for_traffic_project_assignment\training_dataset.csv", sep=';')
ev_ds = pd.read_csv(r"C:\Users\playm\OneDrive\Skrivbord\Labbar - Transport\dataset_for_traffic_project_assignment\evaluation_dataset.csv", sep=';')
f_ev_ds = pd.read_csv(r"C:\Users\playm\OneDrive\Skrivbord\Labbar - Transport\dataset_for_traffic_project_assignment\final_evaluation_dataset.csv", sep=';')


In [ ]:
print('\n training dataset')
print(tds.head())
print('\n evaluation dataset------------')
print(ev_ds.head())
print('\n final evaluation dataset --------')
print(f_ev_ds.head())

In [ ]:
print('Training dataset')
print(tds.isna().sum())
print('\n eval dataset')
print(ev_ds.isna().sum())
print('\n final eval dataset')
print(f_ev_ds.isna().sum())

In [ ]:
#drop all NaN values.
f_ev_ds_noNAN = f_ev_ds.dropna()
tds_noNAN = tds.dropna()
ev_ds_noNAN = ev_ds.dropna()

In [ ]:
tds_noNAN.head(10)

In [ ]:
#Filter all of the datasets so we only have data from the correct portal / sensor group
#This will ensure we can get more reliable data as the predicitons just do not work when they are all together.

portal_of_interest = "E4S 56,780"
tds_noNAN = tds_noNAN[tds_noNAN['PORTAL'] == portal_of_interest]
ev_ds_noNAN = ev_ds_noNAN[ev_ds_noNAN['PORTAL'] == portal_of_interest]
f_ev_ds_noNAN = f_ev_ds_noNAN[f_ev_ds_noNAN['PORTAL'] == portal_of_interest]
print(f"Training rows for {portal_of_interest}: {len(tds_noNAN)}")
print(f"Evaluation rows for {portal_of_interest}: {len(ev_ds_noNAN)}")
print(f"Final Evaluation rows for {portal_of_interest}: {len(f_ev_ds_noNAN)}")

In [ ]:
print(f'Org dataset amount:{len(tds)}')
print(f'Org dataset amount:{len(ev_ds)}')
print(f'Org dataset amount:{len(f_ev_ds)}')

print(f'Dropped Dataset amount:{len(tds_noNAN)}')
print(f'Dropped Dataset amount:{len(ev_ds_noNAN)}')
print(f'Dropped Dataset amount:{len(f_ev_ds_noNAN)}')

# Forts.

In [ ]:
#define what is congestion and what is not.
# congestion_flow = tds[tds['FLOW'] < 100]
# congestion_speed = tds[tds['SPEED_MS_AVG'] < 10]

ev_ds_noNAN['datetime'] = pd.to_datetime(ev_ds_noNAN['Date'].astype(str) + ' ' + ev_ds_noNAN['Time'])

ev_ds_offpeak = ev_ds_noNAN[(ev_ds_noNAN['datetime'].dt.time >= pd.to_datetime("04:00:00").time()) &
                            (ev_ds_noNAN['datetime'].dt.time <= pd.to_datetime("06:00:00").time())]
ev_ds_onpeak = ev_ds_noNAN[(ev_ds_noNAN['datetime'].dt.time >= pd.to_datetime("07:30:00").time()) &
                           (ev_ds_noNAN['datetime'].dt.time <= pd.to_datetime("08:30:00").time())]

ev_ds_offpeak = ev_ds_offpeak[(ev_ds_offpeak['FLOW'] > 0) &
                              (ev_ds_offpeak['SPEED_MS_AVG'].between(5, 50))]
ev_ds_onpeak = ev_ds_onpeak[(ev_ds_onpeak['FLOW'] > 0) &
                            (ev_ds_onpeak['SPEED_MS_AVG'].between(5, 50))]


In [ ]:
# converting the date and time columns int o just one "datetime" coluinm
tds_noNAN['datetime'] = pd.to_datetime(tds_noNAN['Date'].astype(str) + ' ' + tds_noNAN['Time'])

# filtering the dataset to only include the data within hours 04:00 and 06:00, this to get an accurate description of free flow speed.
tds_offpeak = tds_noNAN[(tds_noNAN['datetime'].dt.time >= pd.to_datetime("04:00:00").time()) &
                (tds_noNAN['datetime'].dt.time <= pd.to_datetime("06:00:00").time())]

tds_onpeak= tds_noNAN[(tds_noNAN['datetime'].dt.time >= pd.to_datetime("07:30:00").time()) &
                  (tds_noNAN['datetime'].dt.time <= pd.to_datetime("08:30:00").time())]
#Ensuring we only get correctly read speeds. E.g Removing any negative speeds and random slow drivers etc
#that do not actually depict the actual free flowspeed e.g people speeding and drinving super slow...
tds_offpeak = tds_offpeak[(tds_offpeak['FLOW'] > 0) &
                        (tds_offpeak['SPEED_MS_AVG'].between(2, 50))]
tds_onpeak = tds_onpeak[(tds_onpeak['FLOW'] > 0) &
                        (tds_onpeak['SPEED_MS_AVG'].between(2, 50))]

#  Modeling

In [ ]:
#defining what congestion is for off peak hours
ffs_kmh = 85 #Through basic theory, a 4 lane road with a speed limit of 70 km/h, would have the free flow speed around 85 km/h
ffs_ms = ffs_kmh * 0.277778 # Converting km/h to m/s
cong_th = (ffs_kmh * 0.7) # We use mild congestion as a staple, as otherwise the dataset for congested times is just to small to work with.
print(f"Free flow speed in [km/h]: {ffs_kmh}")
print(f"Free flow speed in [m/s]: {ffs_ms}")
print(f"Congestion Speed Threshold [km/h]: {cong_th}")

#hard coding cong as 60 [km/h] for simplicty
cong_th_2 = 61

In [ ]:
#defining what congestion is for on peak hours
ffs_kmh_2 = 85 #Through basic theory, a 4 lane road with a speed limit of 70 km/h, would have the free flow speed around 85 km/h
ffs_ms_2 = ffs_kmh_2 * 0.277778 # Converting km/h to m/s
cong_th_2 = (ffs_kmh_2 * 0.7) # We use mild congestion as a staple, as otherwise the dataset for congested times is just to small to work with.
print(f"Free flow speed in [km/h]: {ffs_kmh_2}")
print(f"Free flow speed in [m/s]: {ffs_ms_2}")
print(f"Congestion Speed Threshold [km/h]: {cong_th_2}")

#hard coding cong as 61 [km/h] for simplicty and to catch those going around 60km/h for a fairer comparison
cong_th_2 = 61

In [ ]:
print(cong_th_2)
tds_onpeak.info()

In [ ]:
#Creat our rolling and shifting.
tds_onpeak['pred15'] = tds_onpeak['SPEED_MS_AVG'].shift(-1).rolling(15).mean()
tds_onpeak['congestion'] = (tds_onpeak['pred15']*3.6 <= cong_th_2).astype(int)

In [ ]:
tds_onpeak["speed_norm"] = (
    tds_onpeak.groupby("DP_ID")["SPEED_MS_AVG"]
    .transform(lambda x: (x - x.mean()) / x.std())
)

ev_ds_onpeak["speed_norm"] = (
    ev_ds_onpeak.groupby("DP_ID")["SPEED_MS_AVG"]
    .transform(lambda x: (x - x.mean()) / x.std())
)

# Methods for classification

## pca Version

In [ ]:
tds_pca, error_threshold = pca_anomaly_labeling(tds_onpeak, n_components=2, contamination=0.15)
print(f"PCA anomaly threshold: {error_threshold:.6f}")
print(tds_pca["pca_congestion"].value_counts())


## Clustering Version

In [ ]:
tds_cluster, kmeans_model, congested_cluster = kmeans_congestion_labeling(tds_onpeak, n_clusters=3)
print("Congested cluster:", congested_cluster)
print(tds_cluster["cluster_congestion"].value_counts())


In [ ]:
#visualisation of clusters
plt.figure(figsize=(8,5))
sns.scatterplot(
    data=tds_cluster,
    x=tds_cluster["SPEED_MS_AVG"]*3.6,  # convert to km/h
    y=tds_cluster["FLOW"],
    hue=tds_cluster["cluster"],
    palette="viridis",
    alpha=0.6
)
plt.xlabel("Speed (km/h)")
plt.ylabel("Flow (veh/h)")
plt.title("Traffic states discovered by K-means clustering")
plt.legend(title="Cluster")
plt.show()


## DBSCAN Method

In [ ]:
tds_db, db_model = dbscan_congestion_labeling(tds_onpeak, eps=0.25, min_samples=20)
print(tds_db["db_congestion"].value_counts())


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
sns.scatterplot(
    x=tds_db["SPEED_MS_AVG"] * 3.6,  # convert m/s → km/h
    y=tds_db["FLOW"],
    hue=tds_db["db_congestion"],
    palette={0:"skyblue", 1:"red"},
    alpha=0.6
)
plt.xlabel("Speed (km/h)")
plt.ylabel("Flow (veh/h)")
plt.title("DBSCAN-based congestion labeling (red = anomalous/congested)")
plt.show()


# Hard constrained function

In [ ]:
#function where the features are created and..
def make_features_targets(df, horizon=15, window=15, cong_th_2=61):
    df = df.copy()
    df = df.sort_values("datetime")

    # Ensure so the df is in datetime and not Date and Time..
    if "datetime" not in df.columns:
        df["datetime"] = pd.to_datetime(df["Date"].astype(str) + " " + df["Time"])

    # Rolling past features
    df["past_mean_speed"] = df["SPEED_MS_AVG"].rolling(window).mean()
    df["past_std_speed"] = df["SPEED_MS_AVG"].rolling(window).std()
    df["past_min_speed"] = df["SPEED_MS_AVG"].rolling(window).min()

    df["past_mean_flow"] = df["FLOW"].rolling(window).mean()
    df["past_std_flow"] = df["FLOW"].rolling(window).std()

    #
    df["past_speed_trend"] = df["SPEED_MS_AVG"] - df["SPEED_MS_AVG"].rolling(window).mean()
    df["flow_to_speed_ratio"] = df["past_mean_flow"] / (df["past_mean_speed"] + 1e-3)

    # Share of low-speed observations
    df["share_low_speed"] = (
        (df["SPEED_MS_AVG"] * 3.6 < cong_th_2)
        .rolling(window)
        .mean()
    )

    # tem.poral features.
    df["hour"] = df["datetime"].dt.hour
    df["month"] = df["datetime"].dt.month
    df["dayofweek"] = df["datetime"].dt.dayofweek

    df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
    df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)
    df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
    df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)
    df["dow_sin"] = np.sin(2 * np.pi * df["dayofweek"] / 7)
    df["dow_cos"] = np.cos(2 * np.pi * df["dayofweek"] / 7)

    # target, future congestion. Hard constrained to our cong_th_2
    future_speed = df["SPEED_MS_AVG"].shift(-horizon).rolling(horizon).mean()
    df["future_congestion"] = (future_speed * 3.6 < cong_th_2).astype(int)

    # Speed and flow deltas
    df["speed_delta_5min"] = df["SPEED_MS_AVG"] - df["SPEED_MS_AVG"].shift(5)
    df["flow_delta_5min"] = df["FLOW"] - df["FLOW"].shift(5)

    # Rolling slope of speed
    df["speed_slope"] = (
    df["SPEED_MS_AVG"].rolling(5)
      .apply(lambda x: np.polyfit(range(len(x)), x, 1)[0], raw=False)
    )

    # dropping eventual nans created during the process
    df = df.dropna().reset_index(drop=True)

    # testing feature block.
    # features = [
        # "past_mean_speed", "past_std_speed", "past_min_speed",
        # "past_mean_flow", "past_std_flow",
        # "past_speed_trend", "flow_to_speed_ratio",
        # "share_low_speed",
        # "hour_sin", "hour_cos",
        # "month_sin", "month_cos",
        # "dow_sin", "dow_cos",'speed_slope','speed_delta_5min','flow_delta_5min']

# these are the only features used
    features = [
        "past_mean_speed", "past_std_speed", "past_min_speed",
        "past_speed_trend",
        "share_low_speed",
        "hour_sin", "hour_cos",
        "month_sin", "month_cos",
        "dow_sin", "dow_cos",'speed_slope','speed_delta_5min','flow_delta_5min'
        ]

    target = "future_congestion"

    return df, features, target


## Initate the methods

Run only one, aka the chosen method code block.

In [ ]:
tds_fixad, features, target = make_features_targets(tds_onpeak, horizon=15, cong_th_2=cong_th_2)

# Create evaluation data (for 2022)
ev_fixad, _, _ = make_features_targets(ev_ds_onpeak, horizon=15, cong_th_2=cong_th_2)

In [ ]:
tds_pca, error_threshold = pca_anomaly_labeling(tds_onpeak, n_components=2, contamination=0.15)
tds_fixad, features, target = make_features_targets(tds_pca, horizon=15)
tds_fixad["future_congestion"] = tds_pca["pca_congestion"].iloc[:len(tds_fixad)].values


In [ ]:
tds_cluster, _, _ = kmeans_congestion_labeling(tds_onpeak, n_clusters=3)
tds_fixad, features, target = make_features_targets(tds_cluster, horizon=15)
tds_fixad["future_congestion"] = tds_cluster["cluster_congestion"].iloc[:len(tds_fixad)].values


In [ ]:
tds_db, db_model = dbscan_congestion_labeling(tds_onpeak, eps=0.25, min_samples=20)
tds_fixad, features, target = make_features_targets(tds_db, horizon=15)
tds_fixad["future_congestion"] = tds_db["db_congestion"].iloc[:len(tds_fixad)].values


# Dela upp i train test splits.

In [ ]:
#Split into train / test sets
X = tds_fixad[features].values
y = tds_fixad[target].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
#Scaling the datat.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Test data models

In [ ]:
from sklearn.linear_model import LogisticRegression

#Logistic regression model.
lr = LogisticRegression(random_state = 42, max_iter = 1000, class_weight='balanced')
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)
y_prob_lr = lr.predict_proba(X_test_scaled)[:, 1]

print("Logistic Regression Dataset")
print(confusion_matrix(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr, digits=3))
print(f" The roc_auc score is: {roc_auc_score(y_test, y_prob_lr)}")

In [ ]:
print("Class distribution in y_train:")
print(pd.Series(y_train).value_counts())

In [ ]:
print("Class distribution in tds_fixad['future_congestion']:")
print(tds_fixad['future_congestion'].value_counts())

In [ ]:
from xgboost import XGBClassifier
#Xgboos classifier.
scale_pos_weight = (len(y_train) - sum(y_train)) / sum(y_train)
xgb_clf = XGBClassifier(
    n_estimators=500,
    learning_rate=0.03,
    max_depth=3,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1
    )

xgb_clf.fit(X_train_scaled, y_train)

# Predict.
y_pred_xgb = xgb_clf.predict(X_test_scaled)
y_prob_xgb = xgb_clf.predict_proba(X_test_scaled)[:, 1]


# stats
print("XGBoost Training dataset")
print(confusion_matrix(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb, digits=3))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(
    n_estimators=400,
    max_depth=2,
    class_weight='balanced',  # handle the class imbalance
    random_state=42,
    n_jobs=-1 #n_jobs help using all cores.
    )

rf_clf.fit(X_train_scaled, y_train)
y_pred_rf = rf_clf.predict(X_test_scaled)
#stats
print("Random Forest Training dataset")
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf, digits=3))

In [ ]:
rf_clf = RandomForestClassifier(
    n_estimators=400,
    max_depth=8,
    min_samples_split=5,
    min_samples_leaf=3,
    max_features='sqrt',
    class_weight={0:1, 1:2},
    criterion='entropy',
    random_state=42,
    n_jobs=-1
)
rf_clf.fit(X_train_scaled, y_train)
y_prob_rf = rf_clf.predict_proba(X_test_scaled)[:, 1]
y_pred_rf = (y_prob_rf >= 0.4).astype(int)


In [ ]:
print("Random Forest Training dataset")
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf, digits=3))

In [ ]:
from sklearn.svm import SVC
#Support vector classifier.
svm = SVC(
    kernel='rbf',
    probability=True,
    C=0.5,
    gamma='scale',
    random_state=42,
    class_weight={0: 1, 1: 2}
    )

svm.fit(X_train_scaled, y_train)
y_pred_svm = svm.predict(X_test_scaled)
y_prob_svm = svm.predict_proba(X_test_scaled)[:, 1]

#stats

In [ ]:
print('SVM Training Dataset')
print(confusion_matrix(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm, digits=3))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Initialize and train the KNN classifier
knn_clf = KNeighborsClassifier(n_neighbors=5, weights='distance', metric='minkowski', n_jobs=-1) # You can adjust n_neighbors
knn_clf.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred_knn = knn_clf.predict(X_test_scaled)

# Print evaluation metrics
print("K-Nearest Neighbors Training dataset")
print(confusion_matrix(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn, digits=3))

In [ ]:
print("Logistic Regression Dataset")
print(confusion_matrix(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr, digits=3))
print("\n")

print("XGBoost Training dataset")
print(confusion_matrix(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb, digits=3))
print("\n")
print("Random Forest Training dataset")
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf, digits=3))
print("\n")
print("SVM Training dataset")
print(confusion_matrix(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm, digits=3))
print('\n')
print("K-Nearest Neighbors Training dataset")
print(confusion_matrix(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn, digits=3))

In [ ]:
#create empty list to store results
results = []

#for loop that adds results to the list "results"
for name, pred in [
    ("Logistic Regression", y_pred_lr),
    ("Random Forest", y_pred_rf),
    ("XGBoost", y_pred_xgb),
    ("SVM", y_pred_svm),
    ("K-Nearest Neighbors", y_pred_knn)
]:
    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, pred),
        "Precision": precision_score(y_test, pred),
        "Recall": recall_score(y_test, pred),
        "F1": f1_score(y_test, pred)
    })

results_df = pd.DataFrame(results)
#print the results.
print(results_df)

## GridSearching

For clarity this part was helped by generative AI.

In [ ]:
from sklearn.metrics import (
    f1_score, precision_score, recall_score,
    classification_report, confusion_matrix,
    precision_recall_curve, make_scorer
)

def train_and_evaluate_model(model, model_name, param_grid,
                             X_train_scaled, y_train,
                             X_test_scaled, y_test,
                             scoring_metric='f1',
                             threshold_tuning=True,
                             show_plots=True):
    """
    Trains any model using GridSearchCV, tunes threshold for best F1,
    and prints performance + plots precision-recall curve.
    """

    print(f"\n{'='*80}")
    print(f" Starting Grid Search + Evaluation for {model_name}")
    print(f"{'='*80}")

    # Grid Searching  all models
    scorer = make_scorer(f1_score, pos_label=1)
    grid = GridSearchCV(model, param_grid, scoring=scorer, cv=3, n_jobs=-1, verbose=2)
    grid.fit(X_train_scaled, y_train)

    best_model = grid.best_estimator_
    best_params = grid.best_params_
    print(f"\n Best parameters for {model_name}: {best_params}")
    print(f" Best CV F1-score: {grid.best_score_:.3f}")

    #  probabilities  scores
    if hasattr(best_model, "predict_proba"):
        y_prob = best_model.predict_proba(X_test_scaled)[:, 1]
    elif hasattr(best_model, "decision_function"):
        y_scores = best_model.decision_function(X_test_scaled)
        y_prob = 1 / (1 + np.exp(-y_scores))
    else:
        y_prob = best_model.predict(X_test_scaled)
        threshold_tuning = False
        print("Model has no probability outputs: skipping threshold tuning.")

    # hreshold tuning for best F1
    if threshold_tuning:
        thresholds = np.arange(0.1, 0.9, 0.02)
        f1_scores = [f1_score(y_test, (y_prob >= t).astype(int)) for t in thresholds]
        best_t = thresholds[np.argmax(f1_scores)]
        best_f1 = np.max(f1_scores)
        print(f"Best threshold = {best_t:.2f} (F1 = {best_f1:.3f})")
    else:
        best_t = 0.5

    # Evaluate the models
    y_pred = (y_prob >= best_t).astype(int)
    print("\n📊 Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print(f"\n📋 Classification Report ({model_name}):")
    print(classification_report(y_test, y_pred, digits=3))

    # Stores the results in a list..
    results = {
        "model_name": model_name,
        "best_params": best_params,
        "best_threshold": best_t,
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1": f1_score(y_test, y_pred),
        "model": best_model
    }

    # Plot precision-recall curve for easy visualizxation of the models performances.
    if show_plots and threshold_tuning:
        prec, rec, thr = precision_recall_curve(y_test, y_prob)
        plt.figure(figsize=(6, 5))
        plt.plot(rec, prec, label="Precision–Recall Curve")
        plt.axvline(x=results["recall"], color='r', linestyle='--', label=f"Recall = {results['recall']:.2f}")
        plt.axhline(y=results["precision"], color='g', linestyle='--', label=f"Precision = {results['precision']:.2f}")
        plt.title(f"{model_name} Precision–Recall Curve")
        plt.xlabel("Recall")
        plt.ylabel("Precision")
        plt.legend()
        plt.grid(True)
        plt.show()

    return results


In [ ]:
# defining what parameters we are looking for to get the best results.
param_grids = {
    "Logistic Regression": {
        "model": LogisticRegression(max_iter=500, class_weight='balanced', solver='lbfgs', random_state=42),
        "params": {
            "C": [0.1, 1, 5, 10],
            "penalty": ["l2"]
        }
    },

    "Random Forest": {
        "model": RandomForestClassifier(random_state=42, n_jobs=-1),
        "params": {
            "n_estimators": [200, 400],
            "max_depth": [4, 6, 8, None],
            "min_samples_split": [2, 5],
            "min_samples_leaf": [1, 3, 5],
            "class_weight": ["balanced", {0:1, 1:2}],
            "criterion": ["gini", "entropy"]
        }
    },

    "XGBoost": {
        "model": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
        "params": {
            "n_estimators": [200, 400],
            "max_depth": [4, 6, 8],
            "learning_rate": [0.05, 0.1, 0.2],
            "subsample": [0.8, 1.0],
            "colsample_bytree": [0.8, 1.0],
            "scale_pos_weight": [1, 2, 3]
        }
    },

    "Support Vector Machine": {
        "model": SVC(probability=True, random_state=42),
        "params": {
            "C": [0.5, 1, 2],
            "gamma": ["scale", "auto"],
            "kernel": ["rbf", "poly"]
        }
    },

    "K-Nearest Neighbors": {
        "model": KNeighborsClassifier(n_jobs=-1),
        "params": {
            "n_neighbors": [3, 5, 7, 9],
            "weights": ["uniform", "distance"],
            "metric": ["minkowski", "euclidean"]
        }
    }
}


In [ ]:
results = []

for name, setup in param_grids.items():
    res = train_and_evaluate_model(
        setup["model"],
        model_name=name,
        param_grid=setup["params"],
        X_train_scaled=X_train_scaled,
        y_train=y_train,
        X_test_scaled=X_test_scaled,
        y_test=y_test,
        scoring_metric='f1',
        threshold_tuning=True,
        show_plots=True
    )
    results.append(res)

# Comparison table
results_df = pd.DataFrame(results)[["model_name", "precision", "recall", "f1", "best_threshold", "best_params"]]
print("\n Model Comparison Summary:")
print(results_df.sort_values(by="f1", ascending=False).to_string(index=False))


# Evaluation data models

In [ ]:
#call function to fix ev_ds_onpeak dataset to contain the same features.
ev_fixad, _, _ = make_features_targets(ev_ds_onpeak, horizon=15, cong_th_2=cong_th_2)

In [ ]:
#running the models on the evaluation dataset.
X_eval = ev_fixad[features].values
y_eval = ev_fixad[target].values

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_eval_scaled  = scaler.transform(X_eval)

In [ ]:


from sklearn.metrics import confusion_matrix, classification_report

for name, model in [
    ("Logistic Regression", lr),
    ("Random Forest", rf_clf),
    ("XGBoost", xgb_clf),
    ("SVM", svm)
]:
    y_pred_eval = model.predict(X_eval_scaled)
    print(f"\n{name} (Evaluation 2022 dataset):")
    print(confusion_matrix(y_eval, y_pred_eval))
    print(classification_report(y_eval, y_pred_eval, digits=3))


In [ ]:
#loop through models on evaluation dataset.
for name, model in [
    ("Logistic Regression", lr),
    ("Random Forest", rf_clf),
    ("XGBoost", xgb_clf),
    ("SVM", svm)
]:
    y_pred_eval = model.predict(X_eval_scaled)
    print(f"\n{name} (Evaluation 2022 dataset):")
    print(confusion_matrix(y_eval, y_pred_eval))
    print(classification_report(y_eval, y_pred_eval, digits=3))

In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns
# Filter the final evaluation dataset by the portal of interest
f_ev_ds_portal = f_ev_ds[f_ev_ds['PORTAL'] == portal_of_interest].copy()

# Drop NaNs from f_ev_ds_portal before feature engineering
f_ev_ds_cleaned = f_ev_ds_portal.dropna(subset=["SPEED_MS_AVG", "FLOW"]).copy()

# Ensure datetime is present
if "datetime" not in f_ev_ds_cleaned.columns:
    f_ev_ds_cleaned["datetime"] = pd.to_datetime(f_ev_ds_cleaned["Date"].astype(str) + " " + f_ev_ds_cleaned["Time"])

# Filter again for peak hours
f_ev_ds_onpeak = f_ev_ds_cleaned[
    (f_ev_ds_cleaned['datetime'].dt.time >= pd.to_datetime("07:30:00").time()) &
    (f_ev_ds_cleaned['datetime'].dt.time <= pd.to_datetime("08:30:00").time())
].copy()

# # Use the make_features_targets function on the filtered (onpeak) data set for final evaluation.
# f_ev_fixad, features_eval, target_eval = make_features_targets(
#     f_ev_ds_onpeak, horizon=15, cong_th_2=cong_th_2
# )


X_final_eval = f_ev_fixad[features_eval].values
y_final_eval = f_ev_fixad[target_eval].values

#Scaling final evaluation data.
# Scaler for threshold.
scaler_threshold = StandardScaler()
X_train_threshold = tds_fixad[features_eval].values
scaler_threshold.fit(X_train_threshold)
X_final_eval_scaled = scaler_threshold.transform(X_final_eval)


# # Evaluation of traning models on the final evaluation dataset
# print("\nEvaluating models on f_ev_ds (threshold labels):")
# # Use the models trained with the threshold-based labels (lr, rf_clf, xgb_clf from earlier cells)
# models_threshold = [("Logistic Regression", lr), ("Random Forest", rf_clf), ("XGBoost", xgb_clf), ("SVM", svm), ("K-Nearest Neighbors", knn_clf)]

# # Get probability predictions for each model on the final evaluation dataset
# y_prob_final_eval = {}
# for name, model in models_threshold:
#     y_pred_final_eval = model.predict(X_final_eval_scaled)
#     y_prob_final_eval[name] = model.predict_proba(X_final_eval_scaled)[:, 1]

#     print(f"\n{name} (Final Evaluation f_ev_ds with threshold labels):")
#     print(confusion_matrix(y_final_eval, y_pred_final_eval))
#     print(classification_report(y_final_eval, y_pred_final_eval, digits=3))
#     print("F1-score:", round(f1_score(y_final_eval, y_pred_final_eval), 3))

# Visualizations

## Training Data Visualisations.

In [ ]:
#Scatter plot, flow vs speed on training data.
sns.scatterplot(
    data=tds_fixad,
    x='past_mean_flow',
    y='past_mean_speed',
    hue='future_congestion',
    alpha=0.3
)
plt.title('Flow vs Speed (Training Data)')
plt.xlabel('Average Flow (veh/min)')
plt.ylabel('Average Speed (m/s)')
plt.show()

## Evaluation set 1 visualisations

## Final Evaluation set visualisations.


In [ ]:
#percision and recall curves for the final evaluation dataset
plt.figure(figsize=(8, 6))

for name, y_prob in y_prob_final_eval.items():
    prec, rec, thr = precision_recall_curve(y_final_eval, y_prob)
    plt.plot(rec, prec, label=name)

plt.xlabel("Recall (True Positive Rate)")
plt.ylabel("Precision")
plt.title("Precision–Recall Curves – Model Comparison (Final Evaluation f_ev_ds)")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:

# Get feature importances from the trained Random Forest
importances = rf_clf.feature_importances_

# Combine with feature names
feature_importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print(feature_importance_df.head(10))


In [ ]:

# Get feature importances from the trained Random Forest
importances = xgb_clf.feature_importances_

# Combine with feature names
feature_importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print(feature_importance_df.head(10))


In [ ]:
from sklearn.inspection import permutation_importance

result = permutation_importance(rf_clf, X_test_scaled, y_test, n_repeats=10, random_state=42)
sorted_idx = result.importances_mean.argsort()[::-1]

for idx in sorted_idx[:10]:
    print(features[idx], result.importances_mean[idx])


In [ ]:
for res in results:
    model = res["model"]
    threshold = res["best_threshold"]
    name = res["model_name"]

    if hasattr(model, "predict_proba"):
        y_prob_eval = model.predict_proba(X_eval_scaled)[:, 1]
        y_pred_eval = (y_prob_eval >= threshold).astype(int)
    else:
        y_pred_eval = model.predict(X_eval_scaled)

    print(f"\n {name} (Evaluation Dataset 2022 metrics results):")
    print(confusion_matrix(y_eval, y_pred_eval))
    print(classification_report(y_eval, y_pred_eval, digits=3))


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score

def evaluate_on_final_dataset(results_list, X_eval_scaled, y_eval):
    """
    Evaluates all trained models (already tuned) on the final evaluation dataset.
    Uses the stored best threshold per model.
    """
    final_eval_results = []

    print("Evaluating all the tuned models on the Final Evaluation Dataset (2022-2023)")

    for res in results_list:
        model = res["model"]
        threshold = res["best_threshold"]
        name = res["model_name"]

        print(f"\n {name} - Final Evaluation")

        # Predict probabilities (if supported)
        if hasattr(model, "predict_proba"):
            y_prob_eval = model.predict_proba(X_eval_scaled)[:, 1]
            y_pred_eval = (y_prob_eval >= threshold).astype(int)
        elif hasattr(model, "decision_function"):
            y_scores = model.decision_function(X_eval_scaled)
            y_prob_eval = 1 / (1 + np.exp(-y_scores))
            y_pred_eval = (y_prob_eval >= threshold).astype(int)
        else:
            y_pred_eval = model.predict(X_eval_scaled)

        # Metrics
        prec = precision_score(y_eval, y_pred_eval)
        rec = recall_score(y_eval, y_pred_eval)
        f1 = f1_score(y_eval, y_pred_eval)

        print(confusion_matrix(y_eval, y_pred_eval))
        print(classification_report(y_eval, y_pred_eval, digits=3))

        final_eval_results.append({
            "model_name": name,
            "eval_precision": prec,
            "eval_recall": rec,
            "eval_f1": f1
        })

    return pd.DataFrame(final_eval_results)


In [ ]:
# Evaluating all tuned models on the final evaluation dataset
final_eval_df = evaluate_on_final_dataset(results, X_final_eval_scaled, y_final_eval)
print("\n Final Evaluation Results (2022 dataset):")
print(final_eval_df.sort_values(by="eval_f1", ascending=False).to_string(index=False))

# Visualisations

In [ ]:
#Distribution of flows during on peak hours
plt.figure(figsize=(10,6))
plt.hist(tds_onpeak['SPEED_MS_AVG']*3.6, bins=40, color='skyblue', edgecolor='black', alpha=0.7)

plt.axvline(ffs_kmh_2, color='red', linestyle='--', linewidth=2, label=f'85th percentile (FFS = {ffs_kmh_2:.2f} km/h)')
plt.axvline(cong_th_2, color='green', linestyle='--', linewidth=2, label=f'Congestion threshold (70% of FFS = {cong_th_2:.2f} km/h)')
plt.xlabel('Speed (km/h)') # Corrected xlabel
plt.ylabel("Frequency")
plt.title("Distribution of Speed During On-Peak (07:30–08:30)") # Corrected title
# plt.legend() # Removed legend call as no labels are defined
plt.grid(alpha=0.3)
plt.show()

In [ ]:
#dist of speeds during off peak hours.

# Plot histogram
plt.figure(figsize=(10,6))
plt.hist(tds_offpeak['SPEED_MS_AVG']*3.6, bins=40, color='skyblue', edgecolor='black', alpha=0.7)
plt.axvline(ffs_kmh, color='red', linestyle='--', linewidth=2, label=f'85th percentile (FFS = {ffs_kmh:.2f} km/h)')
plt.axvline(cong_th, color='green', linestyle='--', linewidth=2, label=f'Congestion threshold (70% of FFS = {cong_th_2:.2f} km/h)')



plt.xlabel("Speed (km/h)")
plt.ylabel("Frequency")
plt.title("Distribution of Speeds During Off-Peak (04:00–06:00)")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
#plot of relation between speed and flow
plt.figure(figsize=(6, 5))
sns.scatterplot(data=tds_onpeak, x="FLOW", y="SPEED_MS_AVG", alpha=0.4, color="teal")
plt.title("Speed–Flow Relationship")
plt.xlabel("Flow (veh/min)")
plt.ylabel("Speed (m/s)")
plt.tight_layout()
plt.show()

In [ ]:
#correlation heatmap of features
plt.figure(figsize=(10, 8))
corr = tds_fixad.corr(numeric_only=True)
sns.heatmap(corr, cmap='coolwarm', center=0, annot=False)
plt.title("Correlation Heatmap of Engineered Features")
plt.tight_layout()
plt.show()


In [ ]:
tds_noNAN["hour"] = tds_noNAN["datetime"].dt.hour
hourly_stats = tds_noNAN.groupby("hour")[["SPEED_MS_AVG", "FLOW"]].mean().reset_index()

fig, ax1 = plt.subplots(figsize=(8, 5))
ax2 = ax1.twinx()
ax1.plot(hourly_stats["hour"], hourly_stats["SPEED_MS_AVG"], color='tab:blue', label='Speed (m/s)')
ax2.plot(hourly_stats["hour"], hourly_stats["FLOW"], color='tab:orange', label='Flow (veh/min)')
ax1.set_xlabel("Hour of Day")
ax1.set_ylabel("Speed (m/s)", color='tab:blue')
ax2.set_ylabel("Flow (veh/min)", color='tab:orange')
plt.title("Average Hourly Variation in Speed and Flow")
plt.tight_layout()
plt.show()


In [ ]:
# re-running PCA on training sclaed features.
pca = PCA()
pca.fit(X_train_scaled)

# cumulative explained variance
plt.figure(figsize=(8, 5))
plt.plot(
    np.cumsum(pca.explained_variance_ratio_),
    marker='o', linewidth=2, color='teal'
)
plt.title('PCA Explained Variance Ratio', fontsize=14)
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.grid(True)
plt.tight_layout()
plt.show()

# Save